In [ ]:
import textwrap 
import requests
import tkinter as tk
from tkinter import ttk
from prettytable import PrettyTable
from tabulate import tabulate
import json

class CPICGUI:
    def __init__(self, master):
        self.master = master
        master.title("CPIC PGx API GUI")

        self.drug_label = ttk.Label(master, text="Drug Name:")
        self.drug_label.grid(row=0, column=0, padx=2, pady=2, sticky=tk.W)
        self.drug_entry = ttk.Entry(master,width=40)
        self.drug_entry.grid(row=0, column=1, padx=2, pady=2)
        
        self.gene_label = ttk.Label(master, text="Gene Symbol:")
        self.gene_label.grid(row=1, column=0, padx=2, pady=2, sticky=tk.W)
        self.gene_entry = ttk.Entry(master,width=40)
        self.gene_entry.grid(row=1, column=1, padx=2, pady=2)

        self.lookup_button1 = ttk.Button(master, text="Get Phenotype", command=self.fetch_lookupkeys,width=30)
        self.lookup_button1.grid(row=2, column=1, padx=2, pady=2)

        self.lookup_button2 = ttk.Button(master, text="Get Ethnicity", command=self.fetch_ethnicity,width=30)
        self.lookup_button2.grid(row=2, column=2, padx=2, pady=2)

        #lookup keys label
        self.lookup_label = ttk.Label(master, text="Phenotypes or Ethnicities")
        self.lookup_label.grid(row=3, column=0, padx=2, pady=2, sticky=tk.W)
        

        self.lookup_listbox = tk.Listbox(master,height=5, width=41)
        self.lookup_listbox.grid(row=3, column=1, padx=2, pady=2)

        #select lookup key button
        self.sel_lookup_button = ttk.Button(master, text="Select", command=self.select_lookupkey,width=30)
        self.sel_lookup_button.grid(row=3, column=2, padx=2, pady=2)
        
       
        
        #lookupkey label
        self.lookupkey_label = ttk.Label(master, text="Phenotype or Ethnicity")
        self.lookupkey_label.grid(row=4, column=0, padx=2, pady=2, sticky=tk.W)

        self.lookupkey_entry = ttk.Entry(master,width=41)
        self.lookupkey_entry.grid(row=4, column=1,padx=2, pady=2 )

        self.recommend_button = ttk.Button(master, text="Get Recommendations", command=self.get_recommendations,width=30)
        self.recommend_button.grid(row=4, column=2, padx=2, pady=2)

        self.clear_button = ttk.Button(master, text="Clear", command=self.clear_recommendations,width=30)
        self.clear_button.grid(row=5, column=2, padx=2, pady=2)
        # Set column width
        #tree.column('A', width=200)


    def fetch_ethnicity(self):
        choices=['Latino', 'American', 'European', 'Oceanian', 'East Asian',
          'Near Eastern', 'Central/South Asian', 'Sub-Saharan African',
          'African American/Afro-Caribbean', 'Other', 'Mixed Ethnicity',
          'Unknown']
        self.lookup_listbox.delete(0, tk.END)
        for choice in choices:
            self.lookup_listbox.insert(tk.END, choice)
        self.lookupkey_label.config(text = "Ethnicity")
        self.lookup_label.config(text = "Ethnicities")
        self.sel_lookup_button.config(text = "Select Ethnicity")
        #self.sel_lookup_button.config(text = "")
    def clear_recommendations(self):
        for item in self.recommend_table.get_children():
            self.recommend_table.delete(item)
        if(self.recommend_table.winfo_exists()):
            self.recommend_table.destroy()
        if(self.scrollbar_x.winfo_exists()):
            self._scrollbar_x.destroy()
        if(self.scrollbar_y.winfo_exists()):    
            self.scrollbar_y.destroy()
    def select_lookupkey(self):
        self.lookupkey_entry.delete(0, tk.END)
        self.lookupkey_entry.insert(0,self.lookup_listbox.get(self.lookup_listbox.curselection()))
    def fetch_lookupkeys(self):
        self.lookup_listbox.delete(0, tk.END)
        self.lookupkey_label.config(text = "Phenotype")
        self.lookup_label.config(text =  "Phenotypes")
        self.sel_lookup_button.config(text = "Select Phenotype")
        
        gene_symbol = self.gene_entry.get()
        lookupkeys = self.get_lookupkeys(gene_symbol)
        if lookupkeys:
            for lookupkey in lookupkeys:
                gene = list(lookupkey.keys())[0]
                phenotype = list(lookupkey.values())[0]
                self.lookup_listbox.insert(tk.END, f"{gene}: {phenotype}")
        else:
            self.lookup_listbox.insert(tk.END, "No lookupkeys found.")

    def get_lookupkeys(self,gene_symbol):
        url = f"https://api.cpicpgx.org/v1/diplotype?genesymbol=eq.{gene_symbol}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            unique_lookupkeys = []
            for item in data:
                lookupkey = item['lookupkey']
                if lookupkey not in unique_lookupkeys:
                    unique_lookupkeys.append(lookupkey)
            return unique_lookupkeys
        return None
    
    def lookupkey_to_prob_list(self, ethnicity, gene_symbol):

        url = "https://api.cpicpgx.org/v1/gene_result?genesymbol=eq.CYP2C19&frequency=not.is.null&order=result&select=result,frequency"
        response = requests.get(url)
        lookupkey_to_prob = []
        data = response.json()
        for item in data:
            lookupkey = {gene_symbol: item['result']} 
            prob = item['frequency'][ethnicity]
            lookupkey_to_prob.append((lookupkey, prob))
        return lookupkey_to_prob
    
    def get_recommendation(self, drugid, lookupkey, prob=1.0):
        url = f"https://api.cpicpgx.org/v1/recommendation?select=drug(name), guideline(name),* &drugid=eq.{drugid}&lookupkey=cs.{lookupkey}"
        response = requests.get(url)
        if response.ok:
            data = response.json()
            recommendations = [{'Drug': item['drug']['name'],
                                'CYP2C19 Phenotype': lookupkey.split("\"")[3],
                                'Probability of Phenotype': prob, 
                                'Recommendation': item['drugrecommendation'],
                                'Guideline': item['guideline']['name']} for item in data]
            return recommendations
        return None

    def get_recommendations(self):
        drugname= self.drug_entry.get()
        url = f"https://api.cpicpgx.org/v1/drug?name=eq.{drugname}"
        response = requests.get(url)
        if response.ok:
            data = response.json()
            drugid = data[0]['drugid']
        else:
            self.recommend_table.insert('', tk.END, values=("No recommendations found.", "", ""))
        
        if(self.lookupkey_label['text'] == 'Phenotype'):
            self.recommend_label = ttk.Label(self.master, text="Recommendations:")
            self.recommend_label.grid(row=5, column=0, padx=2, pady=2, sticky=tk.W)
            
            self.recommend_table = ttk.Treeview(self.master, columns=('Drug', 'Guideline', 'Recommendation'), show='headings', height=4)
            self.scrollbar_x = ttk.Scrollbar(self.master, orient='horizontal', command=self.recommend_table.xview)
            self.scrollbar_x.grid(row=7, column=0, columnspan=3, sticky='ew', padx=2, pady=2)
            self.recommend_table.configure(xscrollcommand=self.scrollbar_x.set)

            
            self.recommend_table.column('Guideline', width=400, anchor="nw", stretch=True, minwidth=200)
            self.recommend_table.column('Recommendation', anchor="nw",width=400,stretch=True, minwidth=200)
            self.recommend_table.grid(row=6, column=0, columnspan=3, padx=2, pady=2)

            self.recommend_table.heading('Drug', text='Drug')
            self.recommend_table.heading('Guideline', text='Guideline')
            self.recommend_table.heading('Recommendation', text='Recommendation')
            self.scrollbar_y = ttk.Scrollbar(self.master, orient="vertical", command=self.recommend_table.yview)
            self.scrollbar_y.grid(row=6, column=3, sticky="ns")
            self.recommend_table.configure(yscrollcommand=self.scrollbar_y.set)
            
            drugname= self.drug_entry.get()
            url = f"https://api.cpicpgx.org/v1/drug?name=eq.{drugname}"
            response = requests.get(url)
            if response.ok:
                data = response.json()
                drugid = data[0]['drugid']
            else:
                self.recommend_table.insert('', tk.END, values=("No recommendations found.", "", ""))
            lookupkey = self.lookupkey_entry.get()
            lookupkey = '{"'+lookupkey.split(":")[0]+'": "'+lookupkey.split(":")[1][1:]+'"}'
            url = f"https://api.cpicpgx.org/v1/recommendation?select=drug(name), guideline(name),* &drugid=eq.{drugid}&lookupkey=cs.{lookupkey}"
            response = requests.get(url)
           

            if response.ok:
                data = response.json()
                recommendations = [{ 'Drug': item['drug']['name'], 'Guideline': item['guideline']['name'], 'Recommendation': item['drugrecommendation'] } for item in data]
                for rec in recommendations:
                    #                    
                    #values=(rec['Drug'], rec['Guideline'], rec['Recommendation'])
                    #self.recommend_table.insert('', tk.END, values=list(map(wrap, values)))
                    self.recommend_table.insert('', tk.END, values=(rec['Drug'], rec['Guideline'], rec['Recommendation']),tags=("wrapped"))
            else:
                self.recommend_table.insert('', tk.END, values=("No recommendations found.", "", ""))
            #self.recommend_table.see(self.recommend_table.get_children()[-1])
            return None
        else:
            ethnicity = self.lookupkey_entry.get()
            gene_symbol = self.gene_entry.get()
            if ethnicity == 'Unknown':
                print("Enquire about patient ethnicity, or recommend genetic testing for most precise results.")
                return None

            # Mixed ethnicity - no good way to handle this case
            if ethnicity == 'Mixed Ethnicity' or ethnicity == 'Other':
                print("Unfortunately our database does not currently contain data for this ethnicity.")
                print("We recommend genetic testing for most precise results.")
                return None

            # Valid ethnicity entered - now, get list of tuples (lookupkey, probability) for that specific ethnicity
            lookupkey_to_prob = self.lookupkey_to_prob_list(ethnicity, gene_symbol)

            # Iterate through list and get recommendations
            recommendations = []
            for lookupkey, prob in sorted(lookupkey_to_prob, key = lambda k:k[1], reverse = True):

                # Set phenotype probability cutoff as desired for final output, or just output all phenotypes
                #if prob > 0.01:
                recommendations = recommendations + self.get_recommendation(drugid, json.dumps(lookupkey), prob)


            self.recommend_label = ttk.Label(self.master, text="Recommendations:")
            self.recommend_label.grid(row=5, column=0, padx=2, pady=2, sticky=tk.W)
    
            self.recommend_table = ttk.Treeview(self.master, columns=('Drug', gene_symbol+' Phenotype', 'Probability of Phenotype', 'Recommendation','Guideline'), show='headings', height=4)
            self.scrollbar_x = ttk.Scrollbar(self.master, orient='horizontal', command=self.recommend_table.xview)
            self.scrollbar_x.grid(row=7, column=0, columnspan=5, sticky='ew', padx=2, pady=2)
            self.recommend_table.configure(xscrollcommand=self.scrollbar_x.set)

            
            self.recommend_table.column('Guideline', width=400, anchor="nw", stretch=True, minwidth=200)
            self.recommend_table.column('Recommendation', anchor="nw",width=400,stretch=True, minwidth=200)
            self.recommend_table.grid(row=6, column=0, columnspan=5, padx=2, pady=2)

            self.recommend_table.heading('Drug', text='Drug')
            self.recommend_table.heading(gene_symbol+' Phenotype', text=gene_symbol+' Phenotype')
            self.recommend_table.heading('Probability of Phenotype', text='Probability of Phenotype')
            self.recommend_table.heading('Recommendation', text='Recommendation')
            self.recommend_table.heading('Guideline', text='Guideline')
            
            self.scrollbar_y = ttk.Scrollbar(self.master, orient="vertical", command=self.recommend_table.yview)
            self.scrollbar_y.grid(row=6, column=5, sticky="ns")
            self.recommend_table.configure(yscrollcommand=self.scrollbar_y.set)
            
            for rec in recommendations:
                self.recommend_table.insert('', tk.END, values=(rec['Drug'], rec[gene_symbol+' Phenotype'], rec['Probability of Phenotype'], rec['Recommendation'], rec['Guideline']),tags=("wrapped"))

            

def wrap(string, lenght=50):
    return '\n'.join(textwrap.wrap(string, lenght))

def print_table(headers, rows):
    table = PrettyTable(headers)
    for row in rows:
        table.add_row(row)
    print(table)

def get_recommendation(drug_name, lookupkey):
    url = f"https://api.cpicpgx.org/v1/recommendation?select=drugid(drug(name)), guideline(name),* &drug(name)=eq.{drug_name}&lookupkey=cs.{lookupkey}"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        recommendations = [{ 'Drug': item['drug']['name'], 'Guideline': item['guideline']['name'], 'Recommendation': item['drugrecommendation'] } for item in data]
        return recommendations
    else:
        print(f"Error: {response.status_code}\n{response.content}")
        return None

if __name__ == '__main__':
    root = tk.Tk()
    gui = CPICGUI(root)
   
    root.mainloop()
